In [1]:
# GPU 확인
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1351766341809210063
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3660972032
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14041188791744612128
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import os, datetime, json
from os.path import join, basename, dirname
from glob import glob
from functools import reduce
from IPython.display import display
from PIL import Image

from imgaug import augmenters as iaa

In [3]:
path_to_current = os.getcwd()
path_to_dataset = join(path_to_current, 'data')
print(f'{path_to_current = }')

path_to_current = 'D:\\DACON_Anomoly'


In [75]:
df = pd.read_csv(join(path_to_dataset, 'new_train_df2.csv'), index_col=['index'])

display(df)

,file_name,label,class
index,,,
0,10000.png,transistor-good,transistor
1,10001.png,capsule-good,capsule
2,10002.png,transistor-good,transistor
3,10003.png,wood-good,wood
4,10004.png,bottle-good,bottle
...,...,...,...
17232,27232.png,grid-broken,grid
17233,27233.png,grid-broken,grid
17234,27234.png,grid-broken,grid


In [4]:
# df_cable = df[df['class']=='cable']

# display(df_cable)

In [74]:
# 절대 경로로 변환
df.file_name = df.file_name.apply(lambda path: join(path_to_dataset, 'train', path))

# class numbering
categories = {label:i for i, label in enumerate(df['label'].unique())}
print(f"{categories = }")

# class: str -> int
df['label'] = df['label'].apply(lambda label: categories[label])

display(df)

categories = {'transistor-good': 0, 'capsule-good': 1, 'wood-good': 2, 'bottle-good': 3, 'screw-good': 4, 'cable-bent_wire': 5, 'carpet-hole': 6, 'hazelnut-good': 7, 'pill-pill_type': 8, 'cable-good': 9, 'metal_nut-scratch': 10, 'pill-good': 11, 'screw-thread_side': 12, 'zipper-fabric_border': 13, 'leather-good': 14, 'pill-scratch': 15, 'toothbrush-good': 16, 'hazelnut-crack': 17, 'screw-manipulated_front': 18, 'zipper-good': 19, 'tile-good': 20, 'carpet-good': 21, 'metal_nut-good': 22, 'bottle-contamination': 23, 'grid-good': 24, 'zipper-split_teeth': 25, 'pill-crack': 26, 'wood-combined': 27, 'pill-color': 28, 'screw-thread_top': 29, 'cable-missing_cable': 30, 'capsule-squeeze': 31, 'zipper-rough': 32, 'capsule-crack': 33, 'capsule-poke': 34, 'metal_nut-flip': 35, 'carpet-metal_contamination': 36, 'metal_nut-color': 37, 'transistor-bent_lead': 38, 'zipper-fabric_interior': 39, 'leather-fold': 40, 'tile-glue_strip': 41, 'screw-scratch_neck': 42, 'screw-scratch_head': 43, 'hazelnut-cut

,file_name,label,class
index,,,
0,D:\DACON_Anomoly\data\train\10000.png,0,transistor
1,D:\DACON_Anomoly\data\train\10001.png,1,capsule
2,D:\DACON_Anomoly\data\train\10002.png,0,transistor
3,D:\DACON_Anomoly\data\train\10003.png,2,wood
4,D:\DACON_Anomoly\data\train\10004.png,3,bottle
...,...,...,...
17232,D:\DACON_Anomoly\data\train\27232.png,55,grid
17233,D:\DACON_Anomoly\data\train\27233.png,55,grid
17234,D:\DACON_Anomoly\data\train\27234.png,55,grid


In [6]:
x_train_all, y_train_all = df.file_name.values, df['label'].values
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all,
                                                  test_size=.1, random_state=0)
print(f'{len(x_train) = }\n{len(x_val) = }')

len(x_train) = 15513
len(x_val) = 1724


In [43]:
input_shape = (224, 224)

# 과적합 방지를 위해서 매번 입력 이미지를 변형한다.
seq = iaa.Sequential([
    iaa.Affine(translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)},
               scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}),
    iaa.Rotate(rotate=(-180, 180)),
])

def normalize(x):
    return x / 255

# 과적합 방지를 위해서 매번 입력 이미지를 변형한다.
seq = iaa.Sequential([
    iaa.Affine(translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)},
               scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}),
    iaa.Rotate(rotate=(-35, 35)),
])

def steps(x, batch_size):
    return (len(x) // batch_size) + 1 if len(x) % batch_size else len(x) // batch_size

def images2arrs(x, input_shape):
    arrs = []
    for image in x:
        arr = np.array(Image.open(image).resize(input_shape), dtype=np.uint8)
        # Grayscale -> RGB
        if len(arr.shape) == 2:
            arr = np.tile(arr[:, :, np.newaxis], reps=[1, 1, 3])
            
        arrs.append(arr)
        
    return np.array(arrs, dtype=np.float32)  # (batch_size,height,width,depth)

# 메모리 부족현상 방지를 위해서 제너레이터 사용한다.
def x_y_generator(x, y, batch_size=64, epochs=1, transform=False, input_shape=(224, 224), shuffle=True):
    seq = iaa.Sequential([
        iaa.Affine(translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)},
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}),
        iaa.Rotate(rotate=(-35, 35)),
    ])
        
    steps_ = steps(x, batch_size)
    batch_order = np.arange(steps_)
    while epochs:
        if shuffle:
            np.random.shuffle(batch_order)  # 배치들간의 순서 섞기
        for order in batch_order:
            start = order * batch_size
            end   = start + batch_size
            
            images = images2arrs(x[start:end], input_shape=input_shape)
            # images = np.array([
            #     np.array(Image.open(image).resize(input_shape), dtype=np.float32) for image in x[start:end]
            # ])

            yield (
                normalize(seq(images=images)) if transform else normalize(images),
                y[start:end],
            )
        epochs -= 1

In [8]:
classes = len(df['label'].unique())

model = tf.keras.applications.vgg19.VGG19(
    include_top=False,          # Not include fully-connected layer
    weights='imagenet',         # 미리 학습된 데이터 사용
    input_tensor=None,
    input_shape=(*input_shape, 3),     # 224 x 224 이미지 사용
    pooling=None,
    classes=classes,            # 모든 품종의 클래스 사용
)

# 모델 커스터마이징
x = model.output
x = GlobalAveragePooling2D(name = "avg_pool")(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(classes, activation='softmax')(x)

model = tf.keras.Model(model.input, predictions)

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,  metrics=['accuracy'])

80142336/80134624 [==============================] - 7s 0us/step


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
batch_size = 16
steps_per_epoch = steps(x_train, batch_size)
validation_steps = steps(x_val, batch_size)
epochs = 20

In [11]:
log_dir = join(path_to_current, 'vgg_logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

# 폴더 생성 함수
mkdirs = lambda dir: os.makedirs(dir) if not os.path.exists(dir) else None
mkdirs(log_dir)

print(log_dir)

checkpoint = ModelCheckpoint(join(log_dir, 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'),
                             monitor='val_loss', save_weights_only=True, save_best_only=True, period=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1,
                               restore_best_weights=True)

D:\DACON_Anomoly\vgg_logs\20220505-075408


In [12]:
history_ = model.fit(x_y_generator(x_train, y_train, batch_size=batch_size, epochs=epochs, transform=True),
                     epochs=epochs, steps_per_epoch=steps_per_epoch,
                     validation_data=x_y_generator(x_val, y_val, batch_size=batch_size, epochs=-1, transform=True),
                     validation_batch_size=batch_size, validation_steps=validation_steps,
                     callbacks=[checkpoint, early_stopping])

Epoch 1/20
970/970 [==============================] - 871s 889ms/step - loss: 2.0339 - accuracy: 0.3438 - val_loss: 1.2808 - val_accuracy: 0.4988
Epoch 2/20
970/970 [==============================] - 841s 868ms/step - loss: 1.1668 - accuracy: 0.5574 - val_loss: 0.9744 - val_accuracy: 0.6218
Epoch 3/20
970/970 [==============================] - 843s 870ms/step - loss: 2.3595 - accuracy: 0.3822 - val_loss: 0.9885 - val_accuracy: 0.6172
Epoch 4/20
970/970 [==============================] - 836s 862ms/step - loss: 0.8980 - accuracy: 0.6631 - val_loss: 0.8332 - val_accuracy: 0.6885
Epoch 5/20
970/970 [==============================] - 820s 846ms/step - loss: 0.7269 - accuracy: 0.7354 - val_loss: 0.7239 - val_accuracy: 0.7210
Epoch 6/20
970/970 [==============================] - 817s 843ms/step - loss: 0.5945 - accuracy: 0.7856 - val_loss: 0.5143 - val_accuracy: 0.8150
Epoch 7/20
970/970 [==============================] - 819s 845ms/step - loss: 0.5095 - accuracy: 0.8187 - val_loss: 0.4826 -

In [84]:
df = pd.read_csv(join(path_to_dataset, 'new_train_df2.csv'), index_col=['index'])

display(df)

,file_name,label,class
index,,,
0,10000.png,transistor-good,transistor
1,10001.png,capsule-good,capsule
2,10002.png,transistor-good,transistor
3,10003.png,wood-good,wood
4,10004.png,bottle-good,bottle
...,...,...,...
17232,27232.png,grid-broken,grid
17233,27233.png,grid-broken,grid
17234,27234.png,grid-broken,grid


In [85]:
de_categories = {i:label for i, label in enumerate(df['label'].unique())}
de_categories

{0: 'transistor-good',
 1: 'capsule-good',
 2: 'wood-good',
 3: 'bottle-good',
 4: 'screw-good',
 5: 'cable-bent_wire',
 6: 'carpet-hole',
 7: 'hazelnut-good',
 8: 'pill-pill_type',
 9: 'cable-good',
 10: 'metal_nut-scratch',
 11: 'pill-good',
 12: 'screw-thread_side',
 13: 'zipper-fabric_border',
 14: 'leather-good',
 15: 'pill-scratch',
 16: 'toothbrush-good',
 17: 'hazelnut-crack',
 18: 'screw-manipulated_front',
 19: 'zipper-good',
 20: 'tile-good',
 21: 'carpet-good',
 22: 'metal_nut-good',
 23: 'bottle-contamination',
 24: 'grid-good',
 25: 'zipper-split_teeth',
 26: 'pill-crack',
 27: 'wood-combined',
 28: 'pill-color',
 29: 'screw-thread_top',
 30: 'cable-missing_cable',
 31: 'capsule-squeeze',
 32: 'zipper-rough',
 33: 'capsule-crack',
 34: 'capsule-poke',
 35: 'metal_nut-flip',
 36: 'carpet-metal_contamination',
 37: 'metal_nut-color',
 38: 'transistor-bent_lead',
 39: 'zipper-fabric_interior',
 40: 'leather-fold',
 41: 'tile-glue_strip',
 42: 'screw-scratch_neck',
 43: 'scre

In [86]:
def images2arrs(x, input_shape):
    arrs = []
    for image in x:
        arr = np.array(Image.open(image).resize(input_shape), dtype=np.uint8)
        # Grayscale -> RGB
        if len(arr.shape) == 2:
            arr = np.tile(arr[:, :, np.newaxis], reps=[1, 1, 3])
        
        arr = arr/255  # normalize
            
        arrs.append(arr)
        
    return np.array(arrs, dtype=np.float32)  # (batch_size,height,width,depth)


In [44]:
y_true = images2arrs(x_val,(224,224))

In [45]:
y_true.shape

(1724, 224, 224, 3)

In [47]:
y_pred = model.predict(y_true)

In [48]:
y_pred.shape

(1724, 88)

In [49]:
from sklearn.metrics import f1_score

def f1(y_true, y_pred):
    amax = np.argmax(y_pred, axis=1)
    print(f1_score(y_true, amax, labels=range(len(categories)), average='macro'))


In [50]:
f1(y_val,y_pred)

0.9201034778075374


In [37]:
test_df = pd.read_csv('./data/test_df.csv',index_col=0)
display(test_df)

,file_name
index,
0,20000.png
1,20001.png
2,20002.png
3,20003.png
4,20004.png
...,...
2149,22149.png
2150,22150.png
2151,22151.png


In [38]:
test_df.file_name = test_df.file_name.apply(lambda path: join(path_to_dataset, 'test', path))

In [39]:
display(test_df)

,file_name
index,
0,D:\DACON_Anomoly\data\test\20000.png
1,D:\DACON_Anomoly\data\test\20001.png
2,D:\DACON_Anomoly\data\test\20002.png
3,D:\DACON_Anomoly\data\test\20003.png
4,D:\DACON_Anomoly\data\test\20004.png
...,...
2149,D:\DACON_Anomoly\data\test\22149.png
2150,D:\DACON_Anomoly\data\test\22150.png
2151,D:\DACON_Anomoly\data\test\22151.png


In [41]:
test_list = test_df.file_name.values
test_list

array(['D:\\DACON_Anomoly\\data\\test\\20000.png',
       'D:\\DACON_Anomoly\\data\\test\\20001.png',
       'D:\\DACON_Anomoly\\data\\test\\20002.png', ...,
       'D:\\DACON_Anomoly\\data\\test\\22151.png',
       'D:\\DACON_Anomoly\\data\\test\\22152.png',
       'D:\\DACON_Anomoly\\data\\test\\22153.png'], dtype=object)

In [51]:
test_arr = images2arrs(test_list,(224,224))

In [53]:
test_arr.shape

(2154, 224, 224, 3)

In [54]:
test_pred = model.predict(test_arr)

In [60]:
amax = np.argmax(test_pred, axis=1).tolist()
amax

[86,
 24,
 0,
 62,
 20,
 11,
 1,
 44,
 82,
 58,
 45,
 2,
 32,
 29,
 81,
 23,
 67,
 36,
 21,
 28,
 86,
 49,
 45,
 42,
 15,
 4,
 56,
 29,
 82,
 87,
 4,
 9,
 4,
 13,
 19,
 13,
 82,
 9,
 59,
 35,
 22,
 9,
 29,
 34,
 35,
 20,
 50,
 71,
 19,
 43,
 9,
 0,
 57,
 69,
 19,
 10,
 78,
 54,
 40,
 1,
 75,
 56,
 1,
 65,
 33,
 82,
 0,
 33,
 8,
 72,
 67,
 2,
 44,
 81,
 59,
 19,
 9,
 82,
 9,
 9,
 11,
 2,
 43,
 85,
 0,
 46,
 39,
 78,
 82,
 19,
 82,
 39,
 9,
 22,
 4,
 74,
 0,
 0,
 22,
 22,
 9,
 21,
 11,
 1,
 2,
 23,
 32,
 26,
 0,
 7,
 82,
 42,
 38,
 84,
 19,
 1,
 24,
 11,
 1,
 59,
 4,
 66,
 22,
 0,
 85,
 75,
 81,
 0,
 15,
 4,
 24,
 10,
 11,
 22,
 1,
 67,
 43,
 23,
 26,
 87,
 84,
 7,
 51,
 9,
 19,
 69,
 36,
 19,
 65,
 2,
 19,
 61,
 75,
 11,
 13,
 20,
 50,
 20,
 7,
 72,
 43,
 29,
 43,
 53,
 21,
 40,
 11,
 79,
 19,
 30,
 21,
 54,
 18,
 58,
 9,
 11,
 4,
 25,
 31,
 19,
 82,
 64,
 3,
 53,
 11,
 36,
 41,
 72,
 61,
 75,
 11,
 31,
 13,
 7,
 11,
 43,
 19,
 32,
 13,
 7,
 75,
 9,
 42,
 46,
 11,
 82,
 11,
 11,
 7,
 55

In [77]:
result =[]
for i in amax:
    result.append(de_categories[i])

In [78]:
result

['tile-rough',
 'grid-good',
 'transistor-good',
 'tile-gray_stroke',
 'tile-good',
 'pill-good',
 'capsule-good',
 'hazelnut-cut',
 'leather-color',
 'leather-glue',
 'bottle-broken_large',
 'wood-good',
 'zipper-rough',
 'screw-thread_top',
 'tile-crack',
 'bottle-contamination',
 'grid-bent',
 'carpet-metal_contamination',
 'carpet-good',
 'pill-color',
 'tile-rough',
 'zipper-squeezed_teeth',
 'bottle-broken_large',
 'screw-scratch_neck',
 'pill-scratch',
 'screw-good',
 'pill-faulty_imprint',
 'screw-thread_top',
 'leather-color',
 'wood-liquid',
 'screw-good',
 'cable-good',
 'screw-good',
 'zipper-fabric_border',
 'zipper-good',
 'zipper-fabric_border',
 'leather-color',
 'cable-good',
 'leather-poke',
 'metal_nut-flip',
 'metal_nut-good',
 'cable-good',
 'screw-thread_top',
 'capsule-poke',
 'metal_nut-flip',
 'tile-good',
 'toothbrush-defective',
 'capsule-scratch',
 'zipper-good',
 'screw-scratch_head',
 'cable-good',
 'transistor-good',
 'hazelnut-hole',
 'hazelnut-print',
 

In [79]:
test_df['label'] = result

In [80]:
display(test_df)

,file_name,label
index,,
0,D:\DACON_Anomoly\data\test\20000.png,tile-rough
1,D:\DACON_Anomoly\data\test\20001.png,grid-good
2,D:\DACON_Anomoly\data\test\20002.png,transistor-good
3,D:\DACON_Anomoly\data\test\20003.png,tile-gray_stroke
4,D:\DACON_Anomoly\data\test\20004.png,tile-good
...,...,...
2149,D:\DACON_Anomoly\data\test\22149.png,tile-gray_stroke
2150,D:\DACON_Anomoly\data\test\22150.png,screw-good
2151,D:\DACON_Anomoly\data\test\22151.png,grid-good


In [81]:
sub_df = test_df.loc[:,['label']]
display(sub_df)

,label
index,
0,tile-rough
1,grid-good
2,transistor-good
3,tile-gray_stroke
4,tile-good
...,...
2149,tile-gray_stroke
2150,screw-good
2151,grid-good


In [83]:
sub_df.to_csv('./submission.csv')